In [2]:
from sbcbinaryformat import Streamer, Writer
import numpy as np
import matplotlib.pyplot as plt

from GetEvent import GetEvent

from ana import AcousticT0 
from scipy.signal import firwin, filtfilt
from scipy.optimize import least_squares
import importlib

In [3]:
importlib.reload(AcousticT0)

<module 'ana.AcousticT0' from '/exp/e961/app/users/runze/projects/LAr10Ana/ana/AcousticT0.py'>

In [4]:
TEST_RUN = "/exp/e961/data/SBC-25-daqdata/20250611_1/"
TEST_RUN2 = "/exp/e961/app/users/runze/data/20251120_12/"
TEST_RUN2 = "/exp/e961/app/users/runze/data/20251121_5/"
# TEST_RUN2 = "/exp/e961/data/users/gputnam/SBC-25-daqdata-test/20251103_1/"
TEST_EVT = 4

In [5]:
data = GetEvent(TEST_RUN2, TEST_EVT,strictMode=False)
data.keys()

/exp/e961/app/users/runze/projects/LAr10Ana/GetEvent.py:106: UserWarning: No scintillation file present in the run directory. Data will not be available in the returned dictionary.
  warnings.warn("No scintillation file present in the run directory. Data will not be available in the returned dictionary.")


dict_keys(['acoustics', 'scintillation', 'cam', 'event_info', 'plc', 'slow_daq', 'run_info', 'run_control'])

In [9]:
data["cam"].keys()
data["event_info"].keys()

dict_keys(['loaded', 'run_id', 'event_id', 'ev_exit_code', 'ev_livetime', 'cum_livetime', 'pset_lo', 'pset_hi', 'pset_ramp1', 'pset_ramp_down', 'pset_ramp_up', 'start_time', 'end_time', 'trigger_source'])

In [10]:
print(data["event_info"]["start_time"])
print(data["event_info"]["end_time"])
print(data["event_info"]["trigger_source"])

[1.7637647e+09]
[1.76376478e+09]
['Pressure']


In [74]:
data["acoustics"].keys()

dict_keys(['loaded', 'Range', 'DCOffset', 'Waveforms', 'sample_rate'])

In [75]:
data["acoustics"]["Range"]


array([[ 2000, 10000, 10000, 10000, 10000, 10000, 10000, 10000]],
      dtype=int16)

In [76]:
wvfs = data["acoustics"]["Waveforms"]
wvfs.shape

(1, 8, 950001)

In [77]:
np.float64([[1e2, 1e3, 1e4, 1e5]]).shape

(1, 4)

In [78]:
# plot the first triggered waveform in each channel
# if channel 7 raw reading is ADC -35/1e4
# if channel 7 raw reading is bits, -35/2**15
# MB.PT1101 := AIn.PT1101 * (-35.0* el3052bits);
wvfs_psi = wvfs*(-35/2**16)


In [79]:
piezo0 = wvfs_psi[0, 7, :]
xlimit = [0, 800]
ylimit = [-22.5,-20]
ylimit = [-20,-18]
# time in miliseconds
total_time = len(piezo0)
time_list_ms = [i/1e3 for i in range(0,total_time,1)]

# add low pass filter
# assuming 1 microsecond time resolution 1e6Hz
numtaps = 10000  # filter length (longer = sharper cutoff)
Fs= 1000000 # sampling rate
Fc = 10 # low pass filter in Hz
fir = firwin(numtaps, Fc, window='hamming', fs=Fs)

piezo0_filtered = filtfilt(fir, [1.0], piezo0)
piezoslope0 = (+piezo0_filtered[1:]-piezo0_filtered[:-1])*1e3 # in bar/ms
# plt.plot(time_list_ms,piezo0, label="piezo channel 0")
# plt.plot(time_list_ms,piezo0_filtered, label="piezo channel 0 filtered")
# plt.xlabel("time/miliseconds")
# plt.ylabel("pressure/psi")
# # plt.xlim(xlimit)
# # plt.ylim(ylimit)
# plt.legend()

In [80]:

# #get the slope after filtering
# fig, ax1 = plt.subplots()

# # First axis
# l1 = ax1.plot(time_list_ms[:-1],piezoslope0, label="piezo channel 0 slope",color="blue")

# ax1.set_xlabel("time/miliseconds")
# ax1.set_ylabel("pressure rate(psi/ms)")
# # ax1.tick_params(axis='y', labelcolor="blue")

# # Second axis (shares X)
# ax2 = ax1.twinx()
# l2 = ax2.plot(time_list_ms,piezo0_filtered, label="piezo channel 0 filtered",color="red")
# ax2.set_ylabel("pressure/psi")
# # ax2.set_ylim(ylimit)

# lines = l1 + l2
# labels = [line.get_label() for line in lines]

# ax1.legend(lines, labels)
# plt.legend()

In [81]:

# ending point find the range of first [10ms,400ms]= 2sigma and then if the rate goes above 4 sigma, mark it as raw end
# then went back 10ms from that end 
# starting point, from [10ms:]
# then fitting find t0

starting_indx = 10000
# starting 10ms after data collection
print(piezoslope0[starting_indx])
p_rate_range = abs(max(piezoslope0[starting_indx:40*starting_indx])-min(piezoslope0[starting_indx:40*starting_indx]))

ending_indx = 0
fitting_ending_indx = 0
hardcut_threshold = p_rate_range*2
for i in range(starting_indx,len(piezoslope0),1):
    if piezoslope0[i] > hardcut_threshold:
        ending_indx = i
        fitting_ending_indx = i-10000
        break
print("index",ending_indx,time_list_ms[ending_indx])
pressure_before_fit = piezo0_filtered[starting_indx:ending_indx]
time_fit_range = time_list_ms[starting_indx:ending_indx]

0.0005969246807957518
index 845046 845.046


In [82]:


# add low pass filter
# assuming 1 microsecond time resolution 1e6Hz
numtaps = 10000  # filter length (longer = sharper cutoff)
Fs= 1000000 # sampling rate
Fc = 10 # low pass filter in Hz
fir = firwin(numtaps, Fc, window='hamming', fs=Fs)

slope0_filtered = filtfilt(fir, [1.0], piezoslope0)
# plt.plot(time_list_ms[:-1],piezoslope0, label="piezo slope channel 0")
# plt.plot(time_list_ms[:-1],slope0_filtered, label="piezo channel 0 filtered")
# plt.xlabel("time/miliseconds")
# plt.ylabel("pressure/psi")
# # plt.xlim(xlimit)
# # plt.ylim(ylimit)
# plt.legend()


# fig, ax1 = plt.subplots()

# # First axis
# l1 = ax1.plot(time_list_ms[:-1],piezoslope0, label="piezo channel 0 slope",color="blue")
# ax1.set_xlabel("time/miliseconds")
# ax1.set_ylabel("pressure rate(psi/ms)")
# # ax1.tick_params(axis='y', labelcolor="blue")



# ax1.legend()

In [83]:
# manually choose starting point and end point
for index in range(len(time_list_ms)):
    if time_list_ms[index]>800:
        print(index)
        break
#8e5

800001


In [84]:
def piecewise_with_t(params, x):
    a, t = params
    y = np.where(x < t, 0, a*(x - t))
    return y

def residuals_with_t(params, x, y):
    return piecewise_with_t(params, x) - y

In [85]:
# fit with function
# f =c when x<t0,
# f=a(x-t0)**2+c when x>t0
slope_before_fit = slope0_filtered[starting_indx:fitting_ending_indx]
time_fitting_range = time_list_ms[starting_indx:fitting_ending_indx]
# initial guesses:
a0 = 8e-6
t0 = 100             # t0 initial guess around 100ms
# c0 = np.mean(pressure_before_fit[:100]) # first 100 data average
p0 = [a0, t0]

# optionally set bounds: a>=0 , t within x-range, c in pressure range
bounds = ([0, min(time_fitting_range)], [np.inf, max(time_fitting_range)])

res = least_squares(residuals_with_t, p0, args=(time_fitting_range,  slope_before_fit), bounds=bounds)
a_fit, t_fit = res.x
print("fitted a, t:", a_fit, t_fit)


n = len(res.fun)
p = len(res.x)

# residual variance (reduced chi^2 estimate)
sigma2 = np.sum(res.fun**2) / (n - p)

# covariance matrix
J = res.jac
cov = sigma2 * np.linalg.inv(J.T @ J)

# 1-sigma uncertainties
param_errors = np.sqrt(np.diag(cov))

a_err, t_err = param_errors

print("a =", a_fit, "±", a_err)
print("t =", t_fit, "±", t_err)
# plot
# plt.scatter(time_fitting_range,  slope_before_fit, s=8, alpha=0.5, label='data')
# xs = np.linspace(min(time_fitting_range), max(time_fitting_range), 500)
# plt.plot(xs, piecewise_with_t((a_fit, t_fit), xs), 'r-', lw=2, label='fit')
# plt.axvline(t_fit, color='gray', linestyle='--', label=f't={t_fit:.3f}')
# plt.legend()

fitted a, t: 7.312692339578783e-06 715.4319744650483
a = 7.312692339578783e-06 ± 2.5235156936351963e-08
t = 715.4319744650483 ± 0.23831287732153156
